<a href="https://colab.research.google.com/github/tasn19/RTAI/blob/main/RTAI_CovidCompetition_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI Against Covid 19 Competition**
competition details: https://r7.ieee.org/montreal-sight/ai-against-covid-19/ 

# Initialization

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# To copy the data to colab memory, run everytime the runtime is restarted
!unzip "/content/gdrive/MyDrive/RTAI-data/archive.zip" -d "/content/archive"

In [ ]:
# To see gpu allocation
!nvidia-smi

# Loading Data

In [ ]:
import os, glob
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split

# TODO: mix train and test for the final training(?)

# data path in drive
# train_images = "/content/gdrive/MyDrive/RTAI-data/archive/train/"
# train_metadata = "/content/gdrive/MyDrive/RTAI-data/archive/train.txt"
# test_images = "/content/gdrive/MyDrive/RTAI-data/archive/test/"
# test_metadata = "/content/gdrive/MyDrive/RTAI-data/archive/test.txt"

# data path in colab
train_images_path = "/content/archive/train/"
train_metadata_path = "/content/archive/train.txt"
test_images_path = "/content/archive/test/"
test_metadata_path = "/content/archive/test.txt"
competition_test_path = "/content/archive/competition_test/"

val_ratio = 0.1
image_size = 299

label_dict = {'positive': 1, 'negative': 0}

class CovidDataSet(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = None
        if annotations_file is not None:
            self.img_labels = pd.read_csv(annotations_file, delimiter = " ", names=['patient id', 'filename', 'class', 'data source'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        if self.img_labels is not None:
            self.len = len(self.img_labels)
        else:
            self.len = sum([len(glob.glob(img_dir+s)) for s in ['*.jpg', '*.png', '*.jpeg']])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        # image = read_image(img_path)
        image = Image.open(img_path).convert("RGB")
        label = label_dict[self.img_labels.iloc[idx, 2]]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


class MyTopCropTransform:
    """
    crop the top <ratio> of the image 
    """
    def __init__(self, ratio):
        self.ratio = ratio

    def __call__(self, x):
        c, h, w = x.shape
        top = int(h*self.ratio)
        return transforms.functional.crop(x, top, 0, h-top, w)

train_transform = transforms.Compose([
    transforms.ToTensor(), 
    MyTopCropTransform(0.08), 
    transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
    # transforms.Resize(size=image_size), transforms.CenterCrop(image_size), #keeps the aspect ratio
    transforms.Resize(size=(image_size,image_size)) #doesn't keep the aspect ratio
])
test_transform = transforms.Compose([
    transforms.ToTensor(), 
    MyTopCropTransform(0.08), 
    # transforms.Resize(size=image_size), transforms.CenterCrop(image_size), # keeps the aspect ratio, crops the image
    transforms.Resize(size=(image_size,image_size)) # doesn't keep the aspect ratio
])

# TODO: try rescale and crop
# TODO: try black and white
# TODO: normalize to -1 and 1

train_dataset = CovidDataSet(train_metadata_path, train_images_path, train_transform)
val_dataset = CovidDataSet(train_metadata_path, train_images_path, test_transform)
test_dataset = CovidDataSet(test_metadata_path, test_images_path, test_transform)
# competition_test_dataset = CovidDataSet(None, competition_test_path, None, test_transform)


# splitting the train dataset to train and validation
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size = val_ratio)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
val_dataset = torch.utils.data.Subset(val_dataset, val_indices)


# TO TAKE RANDOM SUBSET OF DATA
# torch.manual_seed(1)
# train_indices = torch.randperm(len(train_dataset)).tolist()
# val_indices = torch.randperm(len(val_dataset)).tolist()
# test_indices = torch.randperm(len(test_dataset)).tolist()

# # take the first {num} items
# train_dataset = torch.utils.data.Subset(train_dataset, train_indices[:100]) # was [:-1000]
# # take the last {num} items
# val_dataset = torch.utils.data.Subset(val_dataset, test_indices[:100]) # was [-1000:]
# # take the last {num} items
# test_dataset = torch.utils.data.Subset(test_dataset, test_indices[:100]) # was [-1000:]



# # Temp data reading and augmentation test


In [ ]:
from PIL import Image
import os
from torchvision import transforms
import torch
from matplotlib import pyplot as plt


path = "/content/archive/train/"
im_path = os.path.join(path,(os.listdir(path))[10])
image_size = 500

im_transforms = transforms.Compose([
    transforms.ToTensor(), 
    MyTopCropTransform(0.08), 
    # transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
    # transforms.Resize(size=image_size), transforms.CenterCrop(image_size), #keeps the aspect ratio
    transforms.Resize(size=(image_size,image_size)), #doesn't keep the aspect ratio
    # transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

image = Image.open(im_path).convert("RGB")
image_tensor = im_transforms(image)

plt.subplot(1,2,1).imshow(image)
plt.title("original")
plt.subplot(1,2,2).imshow(image_tensor.permute(1,2,0))
plt.title("transformed")
plt.show()



# test_paths = [im_path]

# png, jpeg, jpg = True, True, True
# for impath in os.listdir(path):
#   filename, file_extension = os.path.splitext(impath)
#   if file_extension == ".png" and png:
#     test_paths.append(os.path.join(path,impath))
#     png = False
#   if file_extension == ".jpeg" and jpeg:
#     test_paths.append(os.path.join(path,impath))
#     jpeg = False
#   if file_extension == ".jpg" and jpg:
#     test_paths.append(os.path.join(path,impath))
#     jpg = False
#   if png and jpeg and jpg:
#     break

# for im_path in test_paths:
#   image = Image.open(im_path).convert("L")
#   image_tensor = transforms.ToTensor()(image)
#   print("size: {}, bands: {}".format(image.size, image.getbands()))
#   print("tensor size: {}".format(image_tensor.size()))
#   print("max {},  min {}, some pixel {}".format(torch.max(image_tensor), torch.min(image_tensor), image_tensor[0,400,400]))
#   plt.subplot(1,2,1).imshow(image, cmap=plt.cm.gray)
#   plt.title("original")
#   plt.subplot(1,2,2).imshow(image_tensor.permute(1,2,0).squeeze(), cmap=plt.cm.gray)
#   plt.title("transformed")
#   plt.show()



# Model Resnet


In [ ]:
import torch
import torch.nn as nn

from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2

resnet_models = {'resnet18': resnet18, 
                 'resnet34': resnet34, 
                 'resnet50': resnet50, 
                 'resnet101': resnet101,
                 'resnet152': resnet152,
                 'resnext50_32x4d': resnext50_32x4d, 
                 'resnext101_32x8d': resnext101_32x8d,
                 'wide_resnet50_2': wide_resnet50_2, 
                 'wide_resnet101_2': wide_resnet101_2}

class C19ResNet(nn.Module):
    def __init__(self, pretrained=True, model='resnet18', RGB_input=True):
        super().__init__()
        self.backbone = resnet_models[model](pretrained=pretrained)
        self.fc = nn.Linear(in_features=self.backbone.fc.in_features, out_features=1)
        self.RGB_input = RGB_input
        if not RGB_input:
            self.conv = nn.Conv2d(1, 3, 1)
            self.conv.weight.data.fill_(1)
            self.conv.bias.data.fill_(0)
    
    def forward(self, x):
        if not self.RGB_input:
            x = self.conv(x)
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)        
        x = self.backbone.relu(x)       
        x = self.backbone.maxpool(x)  
        
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)        
        x = self.backbone.layer3(x)        
        x = self.backbone.layer4(x)
        
        x = self.backbone.avgpool(x)
        
        x = x.view(x.size(0), self.backbone.fc.in_features)
        x = self.fc(x)
        
        return x

# Model xception

Xception pythorch implementation source: https://github.com/tstandley/Xception-PyTorch/blob/master/xception.py

In [ ]:
""" 
Creates an Xception Model as defined in:
Francois Chollet
Xception: Deep Learning with Depthwise Separable Convolutions
https://arxiv.org/pdf/1610.02357.pdf
This weights ported from the Keras implementation. Achieves the following performance on the validation set:
Loss:0.9173 Prec@1:78.892 Prec@5:94.292
REMEMBER to set your image size to 3x299x299 for both test and validation
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                  std=[0.5, 0.5, 0.5])
The resize parameter of the validation transform should be 333, and make sure to center crop at 299x299
"""
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.nn import init
import torch

__all__ = ['xception']

model_urls = {
    'xception':'https://www.dropbox.com/s/1hplpzet9d7dv29/xception-c0a72b38.pth.tar?dl=1'
}


class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)
        # (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Block(nn.Module):
    def __init__(self, in_filters, out_filters, reps, strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None
        
        self.relu = nn.ReLU(inplace=True)
        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(self.relu)
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))
        
        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1)) #? why? didn't it apply once in the first condition?
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x


class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=1000): # CHANGED from 1000 to 2
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()

        
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3, 2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,728,2,2,start_with_relu=True,grow_first=True)

        self.block4=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block8=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block9=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block10=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block11=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block12=Block(728,1024,2,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(1024,1536,3,1,1)
        self.bn3 = nn.BatchNorm2d(1536)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes) # not using



        #------- init weights --------
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        #-----------------------------

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def xception(pretrained=False,**kwargs):
    """
    Construct Xception.
    """

    model = Xception(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['xception']))
    return model


covid xcpetion: all the xception layers are used with pretrained weights on imagenet, except for the last fully connected layer

In [ ]:
class C19Xception(nn.Module):
    def __init__(self, pretrained=True, drop_out=False):
        super().__init__()
        self.backbone = xception(pretrained=pretrained)
        self.fc = nn.Linear(in_features=self.backbone.fc.in_features, out_features=1)
        self.drop_out = None
        if drop_out:
            self.drop_out = nn.drop_out(p=0.5)

    
    def forward(self, x, get_features=False):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        
        x = self.backbone.conv2(x)
        x = self.backbone.bn2(x)
        x = self.backbone.relu(x)
        
        x = self.backbone.block1(x)
        x = self.backbone.block2(x)
        x = self.backbone.block3(x)
        x = self.backbone.block4(x)
        x = self.backbone.block5(x)
        x = self.backbone.block6(x)
        x = self.backbone.block7(x)
        x = self.backbone.block8(x)
        x = self.backbone.block9(x)
        x = self.backbone.block10(x)
        x = self.backbone.block11(x)
        x = self.backbone.block12(x)
        
        x = self.backbone.conv3(x)
        x = self.backbone.bn3(x)
        x = self.backbone.relu(x)
        
        x = self.backbone.conv4(x)
        x = self.backbone.bn4(x)
        x = self.backbone.relu(x)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        if get_features:
            return x
            
        if self.drop_out is not None:
            x = self.drop_out(x)
        x = self.fc(x)

        return x

C19Xception summary

In [ ]:
model = C19Xception()
print(model)
print("Number of parameters:")
print(sum([p.numel() for p in model.parameters() if p.requires_grad]))
del model

# Evaluating the results


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

def calculate_score(y_true, y_pred, return_separates = False):
  score = 6 * recall_score(y_true, y_pred, pos_label=1)\
    + 5 * recall_score(y_true, y_pred, pos_label=0)\
    + 3 * precision_score(y_true, y_pred, pos_label=1)\
    + 2 * precision_score(y_true, y_pred, pos_label=0)

  if return_separates:
    return score,\
    recall_score(y_true, y_pred, pos_label=1),\
    recall_score(y_true, y_pred, pos_label=0),\
    precision_score(y_true, y_pred, pos_label=1),\
    precision_score(y_true, y_pred, pos_label=0)
  else:
    return score


# y_true = [0,0,0,0,0,1,1,1,1,1]
# y_pred = [1,1,1,0,0,1,0,0,0,0]
# y_wrong = [1,1,1,1,1,0,0,0,0,0]

# print("sp {:.4f}, sn {:.4f}, pp {:.4f}, pn {:.4f}".format(precision_score(y_true, y_pred, pos_label=1),
#                                           precision_score(y_true, y_pred, pos_label=0),
#                                           recall_score(y_true, y_pred, pos_label=1),
#                                           recall_score(y_true, y_pred, pos_label=0)))

# print("max score: {:.4f}, min score: {:.4f}, some score {:.4f}".format(calculate_score(y_true, y_true),
#                                                                       calculate_score(y_true, y_wrong),
#                                                                       calculate_score(y_true, y_pred)))



# Training


In [ ]:
import time 
import copy
import random
 
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

def run_epoch(model, criterion, optimizer, phase):
    all_pred_probs = torch.tensor([])
    all_labels = torch.tensor([])
    running_loss = 0

    if phase == "train":
        model.train()
    else:
        model.eval()
    
    for i, (inputs, labels) in tqdm(enumerate(dataloaders[phase]), 
                                    leave=False, 
                                    total=len(dataloaders[phase])):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(phase == "train"):
            outputs = model(inputs)
            outputs = outputs.squeeze()
            
            loss = criterion(outputs, labels.float())
            
            if phase == "train":
                loss.backward()
                optimizer.step()
                
        all_pred_probs = torch.cat((all_pred_probs, outputs.detach().clone().sigmoid().cpu()))
        all_labels = torch.cat((all_labels, labels.detach().clone().cpu()))
        running_loss += loss.item() * inputs.size(0)
        
        if (i % logging_steps[phase] == 0):# and (i>0):
            avg_loss = running_loss / ((i+1) * batch_sizes[phase])  
            
            print("[{}]: {} | loss : {:.4f} | score : {:.4f}"
            .format(phase, i//logging_steps[phase], 
                    avg_loss, calculate_score(all_labels, (all_pred_probs > 0.5).type(torch.int8))))
            
    epoch_loss = running_loss / dataset_sizes[phase]
    return epoch_loss, all_labels, all_pred_probs


def train_model(model, criterion, optimizer, num_epochs, device="cuda"):
    since = time.time()

    if os.path.exists(CHECKPOINT_PATH):
        print("Loading checkpoint")
        checkpoint = torch.load(CHECKPOINT_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.cuda()
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        startE = checkpoint['epoch']
    else:
        startE = -1

    best_model_wts = copy.deepcopy(model.state_dict())
    epoch_loss, all_labels, all_pred_probs = run_epoch(model, criterion, optimizer, phase='test')
    best_test_score, sp, sn, pp, pn = calculate_score(all_labels, (all_pred_probs > 0.5).type(torch.uint8), return_separates=True)


    loss_history = {"val":[], "train":[]}
    score_history = {"val":{'score':[], 'sp':[], 'sn':[], 'pp':[], 'pn':[]}, 
                     "train":{'score':[], 'sp':[], 'sn':[], 'pp':[], 'pn':[]}}
    
    for epoch in range(startE+1, num_epochs):
        epoch_time = time.time()
        print("epoch {}/{}".format(epoch+1, num_epochs))

        for phase in ["train", "val"]:
            epoch_loss, all_labels, all_pred_probs = run_epoch(model, criterion, optimizer, phase)
            score, sp, sn, pp, pn, = calculate_score(all_labels, (all_pred_probs > 0.5).type(torch.int8), return_separates=True)

            
            print("---[{}] Epoch {}/{} | time: {:.4f} | Loss : {:.4f} | Score: {:.4f} [sp:{:.4f}-sn:{:.4f}-pp:{:.4f}-pn:{:.4f}]"
            .format(phase, epoch+1, num_epochs, 
                    time.time()-epoch_time, epoch_loss, score, sp, sn, pp, pn))
            loss_history[phase].append(epoch_loss)
            score_history[phase]['score'].append(score)
            score_history[phase]['sp'].append(sp)
            score_history[phase]['sn'].append(sn)
            score_history[phase]['pp'].append(pp)
            score_history[phase]['pn'].append(pn)
            
        epoch_loss, all_labels, all_pred_probs = run_epoch(model, criterion, optimizer, phase='test')
        test_score, sp, sn, pp, pn = calculate_score(all_labels, (all_pred_probs > 0.5).type(torch.int8), return_separates=True)
        if test_score >= best_test_score:
            print("************** best model update: Test score: {:.4f} [sp:{:.4f}-sn:{:.4f}-pp:{:.4f}-pn:{:.4f}] **********".format(test_score, sp, sn, pp, pn))
            best_test_score = test_score
            best_model_wts = copy.deepcopy(model.state_dict())
            
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(), 
                    'best_model_state_dict': best_model_wts},
                    CHECKPOINT_PATH)
        
        print("------ Epoch {}/{} finished. Best test score: {:.4f} ----------------------"
        .format(epoch+1, num_epochs, best_test_score))      
        print()

    time_elapsed = time.time() - since
    print(f"training took {time_elapsed} seconds")
    print(f"best score: {best_test_score}")
    model.load_state_dict(best_model_wts)
    
    return model, loss_history, score_history



# Call training

We use weighted BCE loss to account for the imbalanced dataset. This approach gives us control over the sesitivitiy of the final model. We tested 10, 50, and 100 for the positive class loss weight and found 50 resulted in the best overall score.

In [ ]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.optim import Adam

lr = 3e-3 
# lr = 1e-4
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_name = 'xception'
pretrained = True
drop_out = False
BCE_pos_weight = 50

train_batchsize = 32
val_batchsize = 16
test_batchsize = 16


PATH = "/content/gdrive/MyDrive/RTAI/Trained_models"
CLASS_NAME = model_name\
            + "-epochs_" + str(num_epochs)\
            + "-pretrained_" + str(pretrained)\
            + "-batchsize_" + str(train_batchsize)\
            + "-posweight_" + str(BCE_pos_weight)\
            + "-lr_" + str(lr)\
            + "-drop_out_" + str(drop_out)
            #saves the model with the model name WARNING: overwrites previous model is the same model name exists
model_path = os.path.join(PATH, CLASS_NAME)
history_path = os.path.join(PATH, CLASS_NAME+"-history.png")
details_history_path = os.path.join(PATH, CLASS_NAME+"-details_history.png")
CHECKPOINT_PATH = os.path.join(PATH, CLASS_NAME+"-Checkpoint")


train_dataloader = DataLoader(train_dataset, batch_size=train_batchsize,
                              shuffle=True, num_workers=4) # collate_fn=utils.collate_fn, pin_memory=True
val_dataloader = DataLoader(val_dataset, batch_size=val_batchsize,
                              shuffle=True, num_workers=4) # collate_fn=utils.collate_fn, pin_memory=True                              
test_dataloader = DataLoader(test_dataset, batch_size=test_batchsize, 
                             shuffle=False, num_workers=4) # collate_fn=utils.collate_fn, pin_memory=True


dataloaders={'train': train_dataloader, 
             'val': val_dataloader,
             'test': test_dataloader}

logging_steps = {
    "train": len(dataloaders["train"]) // 10 + 1,
    "val": len(dataloaders["val"]) // 10 + 1,
    "test": len(dataloaders["test"]) // 10 + 1
}

dataset_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    "test": len(test_dataset)
}

batch_sizes = {
    "train": train_batchsize,
    "val": val_batchsize,
    "test": test_batchsize
}

if (model_name == "xception"):
    model = C19Xception(pretrained=pretrained)
else:   
    model = C19ResNet(model=model_name, pretrained=pretrained)

model.to(device)

optimizer = Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([BCE_pos_weight]).to(device),reduction='mean') 
# criterion = nn.BCEWithLogitsLoss() 
# can change pos_weight increasing pos_weight increases the sensitivity
#  of the positive class
# https://discuss.pytorch.org/t/weights-in-bcewithlogitsloss/27452

print("train size:", len(train_dataset))
print("val size:", len(val_dataset))
print("test size:", len(test_dataset))




In [ ]:
model, loss_history, score_history = train_model(model, criterion, optimizer, num_epochs, device = device)


# # Unnecessary Check the alocated memory

In [ ]:
# To track allocated memory
torch.cuda.memory_allocated(device=None)

# Plotting the loss and accuracy history, and saving the model

In [ ]:
from matplotlib import pyplot as plt

plt.subplot(2,2,1)
plt.plot(score_history['val']['sp'], label="validation")
plt.plot(score_history['train']['sp'], label="train")
plt.legend()
plt.title("sp")
plt.subplot(2,2,2)
plt.plot(score_history['val']['sn'], label="validation")
plt.plot(score_history['train']['sn'], label="train")
plt.legend()
plt.title("sn")
plt.subplot(2,2,3)
plt.plot(score_history['val']['pp'], label="validation")
plt.plot(score_history['train']['pp'], label="train")
plt.legend()
plt.title("pp")
plt.subplot(2,2,4)
plt.plot(score_history['val']['pn'], label="validation")
plt.plot(score_history['train']['pn'], label="train")
plt.legend()
plt.title("pn")
plt.savefig(details_history_path)
plt.show()

plt.subplot(1,2,1)
plt.plot(score_history['val']['score'], label="validation")
plt.plot(score_history['train']['score'], label="train")
plt.legend()
plt.title("score")
plt.subplot(1,2,2)
plt.plot(loss_history['val'], label="validation")
plt.plot(loss_history['train'], label="train")
plt.legend()
plt.title("loss")
plt.savefig(history_path)
plt.show()


# Saving the model

torch.save(model.state_dict(), model_path)


# Loading and evaluating the saved model

We adjusted the probability threshold for the positive class to get the best competition score. 

Our best trained model's performance on the test set: 

-- *15.960075 (sp: 1, sn: 0.995, pp: 0.995025, pn: 1)*

its performance on the competition set: 

-- *14.94 (sp: 0.88, pp: 1, sn: 0.99 pn: 0.86)* [changed ?] 
    
By reducing the positive class threshold to 0.35 model's performance on the test set became:

-- *15.762621 (sp:1, sn:0.97, pp: 0.970874 pn: 1)* 

its performance on the competition set: 

-- *15.30 (sp: 0.93, pp: 0.99, sn: 0.98, pn: 0.93)* 


In [ ]:
model_name = 'xception'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = "/content/gdrive/MyDrive/RTAI/Trained_models"
# CLASS_NAME = "xception-epochs_50-pretrained_True-batchsize_32-posweight_10" #first submission
# CLASS_NAME = "xception-epochs_30-pretrained_True-batchsize_32-posweight_50-lr_0.003-drop_out_False" #second submission
CLASS_NAME = "xception-epochs_10-pretrained_True-batchsize_32-posweight_50-lr_0.003" #third & fourth & sixth submission
# CLASS_NAME = "xception-epochs_30-pretrained_True-batchsize_32-posweight_100" # fifth submission
# CLASS_NAME = "xception-epochs_10-pretrained_True-batchsize_32-posweight_10-lr_0.003-drop_out_True"

threshold = 0.35

model_path = os.path.join(PATH, CLASS_NAME)
print(model_path)
# model = C19ResNet(model=model_name, pretrained=False)
model = C19Xception(pretrained=False)
model.to(device)
model.load_state_dict(torch.load(model_path))


print("loaded")

epoch_loss, all_labels, all_pred_probs = run_epoch(model, criterion, optimizer, phase='test')
test_score, sp, sn, pp, pn = calculate_score(all_labels, (all_pred_probs > threshold).type(torch.uint8), return_separates=True)

print("Loaded model test score: {:.6f}, sp: {:.6f}, sn: {:.6f}, pp:{:.6f}, pn:{:.6f}".format(test_score, sp, sn, pp, pn))
n, bins, patches = plt.hist(all_pred_probs, 100, facecolor='blue', alpha=0.5)
plt.show()

# get competition images output

In [ ]:
L = os.listdir(competition_test_path)
L.sort(key=lambda x:int(os.path.splitext(x)[0]))
model.to(device)
model.eval()
preds = []
for p in tqdm(L):
    im_path = os.path.join(competition_test_path, p)
    image = Image.open(im_path).convert("RGB")
    image_tensor = test_transform(image)
    image_tensor = torch.unsqueeze(image_tensor, 0).to(device)

    preds.append(model.forward(image_tensor).detach().clone().squeeze())

    

In [ ]:
probs = torch.tensor(preds).sigmoid()

print("probs", probs)
print("probs mean", torch.mean(probs))
print("probs sd", torch.std(probs))
n, bins, patches = plt.hist(probs, 100, facecolor='blue', alpha=0.5)
plt.show()

for p in probs>threshold:
    print(int(p))
